In [229]:
import pandas as pd
import numpy as np

In [230]:
ds_train = pd.read_csv('ds/train.csv')
ds_songs = pd.read_csv('ds/songs.csv')
ds_songs_extra_info = pd.read_csv('ds/song_extra_info.csv')
ds_members = pd.read_csv('ds/members.csv')

### Беру только 10% от датасета, потому что построение предложений идут очень долго =(

In [232]:
ds_train = ds_train.sample(frac=0.15)

In [233]:
X_train = ds_train.sample(frac=0.8)
X_test = ds_train.drop(X_train.index)

### В качестве предложений(sentence) для каждого юзера буду рассматривать список песен, которые он лайкнул. Где айдишки песен - слова для word2vec

In [234]:
X_train_pos = X_train[X_train['target'] == 1]

In [235]:
from tqdm.notebook import tqdm, trange

In [236]:
user_pos_history = [X_train_pos[X_train_pos['msno'] == user]['song_id'].astype(str).tolist() for user in tqdm(users)]

In [237]:
from gensim.models import Word2Vec

In [238]:
lengths = [len(arr) for arr in user_pos_history]
np.mean(lengths)

12.979711071708863

In [239]:
w2v = Word2Vec(window=8, sg=1, hs=0, negative=3, alpha=0.1, min_alpha=0.0001)

In [240]:
w2v.build_vocab(user_pos_history)

In [241]:
w2v.train(user_pos_history, total_examples=len(user_pos_history), epochs=30)

(10704352, 13396230)

In [242]:
ds_songs = ds_songs.merge(ds_songs_extra_info,on='song_id',how='left')

In [243]:
ds_songs.shape

(2296320, 9)

In [244]:
print(w2v)

Word2Vec(vocab=12318, vector_size=100, alpha=0.1)


In [245]:
def get_similars(name):
    sims = w2v.wv.most_similar_cosmul(name)
    data = []
    for song, similarity in sims:
        d = ds_songs[ds_songs['song_id'] == song].to_dict('records')[0]
        d['similarity'] = similarity
        data.append(d)
    return pd.DataFrame(data)

In [246]:
# name = '0ee/D6HjX0jSytUn3v58C0x6zEzN4bcPq3WlqvrG/Xo=' # heathens, Twenty one pilots
name = 'e/upMhPFcXl3Rxq0Us1abr0FcThlZOeqKJhBobmltBU=' # Shape of you
print(name)
get_similars(name)

e/upMhPFcXl3Rxq0Us1abr0FcThlZOeqKJhBobmltBU=


,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,name,isrc,similarity
0,IKMFuL0f5Y8c63Hg9BXkeNJjE0z8yf3gMt/tOxF4QNE=,244976,1609,The Chainsmokers,Andrew Taggart| Shaun Frank| Frederic Kennett|...,Andrew Taggart| Shaun Frank| Frederic Kennett|...,52.0,Closer,USQX91601347,0.792283
1,JiHWUXkERiCPai7GKXVal+Cns8SWC8Ft0+5NKSQdbfU=,234370,465,Austin Mahone,NaN,NaN,52.0,Shake It For Me (feat. 2 Chainz),QMRSZ1601336,0.771102
2,sgHKhG2d57U2o88razl9SyG1nuuMDf0J2BuDhzoI12Y=,194115,1609,The Chainsmokers,Andrew Taggart| Sara Hejellstrom| Nirob Islam,Andrew Taggart| Sara Hejellstrom| Nirob Islam,52.0,All We Know,USQX91602153,0.770967
3,v/3onppBGoSpGsWb8iaCIO8eX5+iacbH5a4ZUhT7N54=,161175,1616|1609,Alan Walker,Alan Walker| Jesper Borgen| Anders Froen| Gunn...,Alan Walker| Jesper Borgen| Anders Froen| Gunn...,52.0,Alone,NOG841617010,0.764039
4,3VkD5ekIf5duJm1hmYTZlXjyl0zqV8wCzuAh3uocfCg=,217730,465,CHARLIE PUTH,NaN,NaN,52.0,We Don't Talk Anymore,USAT21502909,0.758105
5,0b8M2FLEH176y/ng1rK9ftUYDYnA+1NdoCOUeuyBMtU=,221544,1609,The Chainsmokers,Andrew Taggart| Kris Eriksson| Fredrik Häggstam,Andrew Taggart| Kris Eriksson| Fredrik Häggstam,52.0,Paris,USQX91603031,0.756129
6,n+FLlTL+LrNTa6xpoGkESEzlUMb/hPEM1nLq2U6fEp0=,261172,465,Ed Sheeran,Ed Sheeran| Benjamin Levin,NaN,52.0,Castle on the Hill,GBAHS1600462,0.751877
7,XA08z3r0wdQvY3CE0lFDTd/MuAgD7b0XKZnCvo+ylKE=,245109,921,ZAYN| Taylor Swift,Taylor Swift| Jack Antonoff| Sam Dew,NaN,52.0,I Don’t Wanna Live Forever (Fifty Shades Darker),USQ4E1602586,0.751534
8,CsqfeIr3/stOaLTq+Z+Yt6VnPSqPWS2wxxnCKJL86oM=,220917,1616,Martin Garrix| Dua Lipa,Martijn Garritsen| Georgia Ku Overton| Nathani...,Martijn Garritsen| Georgia Ku Overton| Nathani...,52.0,Scared to Be Lonely,NLM5S1600025,0.748292
9,J4qKkLIoW7aYACuTupHLAPZYmRp08en1AEux+GSUzdw=,212750,1616|1609,Alan Walker,Alan Walker| Jesper Borgen| Anders Froen| Gunn...,Alan Walker| Jesper Borgen| Anders Froen| Gunn...,52.0,Faded,NOG841549010,0.746597


In [247]:
song_data_name_author = []
songs_in_dict = list(w2v.wv.key_to_index.keys())
# for song_id in tqdm(w2v.wv.key_to_index.keys()):
#     print(song_id, ds_songs[ds_songs['song_id'] == song_id]['name'], ds_songs[ds_songs['song_id'] == song_id]['artist_name'])
#     song_data_name_author.append((song_id, ds_songs[ds_songs['song_id'] == song_id]['name'], ds_songs[ds_songs['song_id'] == song_id]['artist_name']))

In [248]:
# w2v.wv.get_vector("e/upMhPFcXl3Rxq0Us1abr0FcThlZOeqKJhBobmltBU=",norm=True)

In [249]:
y_train = X_train['target']
y_test = X_test['target']
X_train = X_train.drop(columns=['target'])
X_test = X_test.drop(columns=['target'])

In [250]:
def get_user_embeddings(X, y):
    embeddings = dict()
    count = dict()
    for (index, row), target in tqdm(zip(X_train.iterrows(), y_train)):
        if target == 1:
            user_id = row['msno']
            song_id = row['song_id']
            if song_id in songs_in_dict:
                if user_id not in count:
                    count[user_id] = 0
                if user_id not in embeddings:
                    embeddings[user_id] = np.zeros(100)
                embeddings[user_id] += w2v.wv.get_vector(song_id, norm=True)
                count[user_id] += 1
    for user_id in embeddings.keys():
        embeddings[user_id] /= count[user_id]
    return embeddings    

In [251]:
user_embeddings = get_user_embeddings(X_train, y_train)

In [252]:
def get_user_song_dot(user_id, song_id):
    if user_id not in user_embeddings:
        return 0
    if song_id not in songs_in_dict:
        return 0
    uv = user_embeddings[user_id]
    sv = w2v.wv.get_vector(song_id)
    
    u_n = np.linalg.norm(uv)
    s_n = np.linalg.norm(sv)
    return np.dot(uv / u_n, sv / s_n)

In [253]:
X_test_c = X_test.copy()

In [254]:
target_res = []
for index, row in tqdm(X_test_c.iterrows()):
    user_id = row['msno']
    song_id = row['song_id']
    target_res.append(get_user_song_dot(user_id, song_id))

### Результат хуже, чем у бустинга, вероятно мало данных, и много пропущенных песен

In [255]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, target_res)

0.6304720423902352

# Добавим эти данные к GBDT 

In [256]:
# ds_train = pd.read_csv('ds/train.csv')
# ds_songs = pd.read_csv('ds/songs.csv')
# ds_songs_extra_info = pd.read_csv('ds/song_extra_info.csv')
# ds_members = pd.read_csv('ds/members.csv')

## (Дальше немного пойдет копипаста 1 части)

In [257]:
genres_for_song = ds_songs['genre_ids'].apply(lambda x: [int(v) for v in str(x).split('|') if v != 'nan'])

In [258]:
genres = dict()
for genre_list in genres_for_song:
    for g in genre_list:
        if g not in genres:
            genres[g] = 0
        genres[g] += 1

In [259]:
sorted_genres = {k: v for k, v in reversed(sorted(genres.items(), key=lambda item: item[1]))}
sorted_genres['0'] = -1

In [260]:
def get_most_popular(g_list):
    res = '0'
    for g in g_list:
        if sorted_genres[res] < sorted_genres[g]:
            res = g
    return res

In [261]:
most_pop = []
for genre_list in genres_for_song:
    most_pop.append(get_most_popular(genre_list))

In [262]:
ds_songs['genre_ids'] = most_pop

In [263]:
def leave_first(column):
    items = ds_songs[column].apply(lambda x: [v for v in str(x).split('|')])
    res = [v[0] for v in items]
    ds_songs[column] = res

In [264]:
leave_first('artist_name')
leave_first('composer')
leave_first('lyricist')

In [265]:
ds_train = ds_train.merge(ds_members,on='msno',how='left')
ds_train = ds_train.merge(ds_songs,on='song_id',how='left')
ds_train = ds_train.fillna(0)

In [266]:
ds_train.drop(columns=['registration_init_time','expiration_date','song_length'], inplace=True)

In [267]:
ds_train['language'] = ds_train['language'].astype(int).astype('category')
ds_train.head(3)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,bd,gender,registered_via,genre_ids,artist_name,composer,lyricist,language,name,isrc
0,VVUJ1v6jIgHOY0+sOA7KJ/DGSEBKqQkE1t6yU7DbB48=,sW0WC79FUxNXU+1kq5EH7RorymurH1lwh/RrIx8YVQE=,search,Album more,album,0,5,27,male,9,691,農曆新年慶團圓,nan,nan,3,敲鑼打鼓迎新歳,0
1,cwyJueBC2115jDABgUIs0v0t5W5hu4i4dnEgECWGPM0=,SX9gmzwMCmsZefwdF2ow3ZSOFRNkEyMn1knjgVAYR74=,my library,Local playlist more,local-playlist,0,5,27,female,3,437,N.O.B.U!!!,Mongol800,nan,17,Chiisanakoinouta,JPPO01328879
2,n645vkTOABpu7yxby85uhzpMukJiOxV1xbPMrIM4748=,q5ircV2ov8Z5124cw4UVozRgVb7KshN6Yh89xScUSy8=,radio,Radio,radio,0,1,0,0,4,465,何韻詩 (HOCC),pan,黃偉文,3,詩與胡說,HKC841000127


In [268]:
def get_dots(row):
    user_id = row['msno']
    song_id = row['song_id']
    return get_user_song_dot(user_id, song_id)

In [269]:
ds_train['dot'] = ds_train.apply(lambda row: get_dots(row), axis=1)

In [270]:
X_train=ds_train.sample(frac=0.8)
X_test=ds_train.drop(X_train.index)

In [271]:
y_train = X_train['target']
y_test = X_test['target']
X_train = X_train.drop(columns=['target'])
X_test = X_test.drop(columns=['target'])

In [272]:
ds_train.head(2)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,bd,gender,registered_via,genre_ids,artist_name,composer,lyricist,language,name,isrc,dot
0,VVUJ1v6jIgHOY0+sOA7KJ/DGSEBKqQkE1t6yU7DbB48=,sW0WC79FUxNXU+1kq5EH7RorymurH1lwh/RrIx8YVQE=,search,Album more,album,0,5,27,male,9,691,農曆新年慶團圓,nan,nan,3,敲鑼打鼓迎新歳,0,0.0
1,cwyJueBC2115jDABgUIs0v0t5W5hu4i4dnEgECWGPM0=,SX9gmzwMCmsZefwdF2ow3ZSOFRNkEyMn1knjgVAYR74=,my library,Local playlist more,local-playlist,0,5,27,female,3,437,N.O.B.U!!!,Mongol800,nan,17,Chiisanakoinouta,JPPO01328879,0.0


In [278]:
X_train = X_train.drop(columns=['source_system_tab','source_screen_name','source_type','city','registered_via','isrc','name'])
X_test = X_test.drop(columns=['source_system_tab','source_screen_name','source_type','city','registered_via','isrc','name'])

In [279]:
X_train.head(2)

,msno,song_id,bd,gender,genre_ids,artist_name,composer,lyricist,language,dot
972986,4tuTIArXO3WO8/zS9y2CC34ywEfChjAKUchlKW00DMM=,bnYQnR0batgokZew6w6DcdiWyXaXJDEh6PSCGlaFQUk=,40,female,465,葉蒨文 (Sally Yeh),Angus Tung,Chen Gui Zhu,3,0.000000
388371,ZTjfgsNVhVJwqvPGc/aPXSy2oEwPpQFwHo7JVR3PILg=,HbqkjwnnIt6BtfmvVyXTdPHAsFCYxU74sv8+s1lfIwE=,0,0,2022,RADWIMPS,Yojiro Noda,nan,17,0.482554


In [280]:
from catboost import CatBoostClassifier

In [285]:
cat_features = [0, 1, 3, 4, 5, 6, 7, 8]

In [290]:
clf = CatBoostClassifier(iterations=300, verbose=20, cat_features=cat_features)

In [291]:
clf.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.5500760	total: 1.29s	remaining: 6m 24s
20:	learn: 0.4273674	total: 29s	remaining: 6m 24s
40:	learn: 0.4212130	total: 54.7s	remaining: 5m 45s
60:	learn: 0.4174166	total: 1m 22s	remaining: 5m 24s
80:	learn: 0.4154201	total: 1m 41s	remaining: 4m 35s
100:	learn: 0.4141045	total: 2m	remaining: 3m 56s
120:	learn: 0.4131130	total: 2m 18s	remaining: 3m 25s
140:	learn: 0.4125181	total: 2m 36s	remaining: 2m 56s
160:	learn: 0.4118277	total: 2m 53s	remaining: 2m 29s
180:	learn: 0.4109799	total: 3m 12s	remaining: 2m 6s
200:	learn: 0.4102694	total: 3m 31s	remaining: 1m 44s
220:	learn: 0.4097684	total: 3m 49s	remaining: 1m 22s
240:	learn: 0.4092334	total: 4m 8s	remaining: 1m
260:	learn: 0.4088311	total: 4m 26s	remaining: 39.8s
280:	learn: 0.4083055	total: 4m 44s	remaining: 19.3s
299:	learn: 0.4080059	total: 5m 1s	remaining: 0us


In [292]:
prediction = clf.predict(X_test)

In [293]:
print(roc_auc_score(y_test, prediction))

0.8173728763541608
